In [ ]:
!pip3 install google-cloud-discoveryengine python-dotenv


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv('./.env')

In [ ]:
PROJECT_ID = os.environ.get('PROJECT_ID')
LOCATION=os.environ.get('LOCATION')
DATA_STORE_ID=os.environ.get('DATA_STORE_ID')

In [ ]:
print(DATA_STORE_ID)
print(PROJECT_ID)
print(LOCATION)

In [ ]:
from typing import List

from google.api_core.client_options import ClientOptions
#from google.cloud import discoveryengine_v1beta as discoveryengine
from google.cloud import discoveryengine_v1 as discoveryengine

#https://cloud.google.com/generative-ai-app-builder/docs/reference/rpc/google.cloud.discoveryengine.v1
def search(
    search_query: str,
    project_id = PROJECT_ID,
    location = LOCATION,
    #engine_id = APP_ID,
    data_store_id = DATA_STORE_ID

) -> List[discoveryengine.SearchResponse]:
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
    )

    client = discoveryengine.SearchServiceClient(client_options=client_options)

    serving_config = client.serving_config_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        #engine_id = engine_id,
        serving_config="default_config",
    )

    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(

        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True,
            max_snippet_count = 5
        ),

        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
        ),

        # V1 REST API에는 명시되어있지만 latest Python library에는 빠져있음
        # Latest Python : https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
        # REST API : https://cloud.google.com/generative-ai-app-builder/docs/reference/rest/v1/ContentSearchSpec#ExtractiveContentSpec
        
        #extractive_content_spec = discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
        #    max_extractive_answer_count=1,
        #    max_extractive_segment_count=1,
        #    return_extractive_segment_score=True
        #)
        
    )

    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        #content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    print(response)

    return response

In [ ]:
print(search('라이센스에 대해 알려줘'))

In [ ]:
import json
req_json = {
    "query": "라이센스에 대해 알려줘",
    "pageSize": 10,
    "queryExpansionSpec": {
        "condition": "AUTO"
    },
    "spellCorrectionSpec": {
        "mode": "AUTO"
    },
    "contentSearchSpec": {
        "summarySpec": {
            "summaryResultCount": 5,
            "ignoreAdversarialQuery": True,
            "includeCitations": True
        },
        "snippetSpec": {
            "maxSnippetCount": 1,
            "returnSnippet": True
        },
        "extractiveContentSpec": {
            "maxExtractiveAnswerCount": 1,
            "maxExtractiveSegmentCount": 1,
            "returnExtractiveSegmentScore": True
        }
    }
}
json_data = json.dumps(req_json)

In [ ]:
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search:search" \
-d '{json_data}'